In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression

import torch
import torch.nn as nn
from torchvision import models
from torch.nn.functional import softmax

# Image Pre-Processing

In [17]:
! pip uninstall pillow -y
! pip install --no-cache-dir pillow

Found existing installation: pillow 11.1.0
Uninstalling pillow-11.1.0:
  Successfully uninstalled pillow-11.1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 55.2 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.5 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.


In [18]:
! pip install facenet-pytorch opencv-python pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 89.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 104.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 85.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 855.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 3.6 MB/s eta 0:00:000:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 17.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━

In [19]:
import os
import cv2
from PIL import Image
from tqdm import tqdm
from torchvision import transforms
from facenet_pytorch import MTCNN, InceptionResnetV1
from torch.nn.functional import cosine_similarity

In [20]:
VIDEO_DIR = '/kaggle/input/meld-dataset/MELD-RAW/MELD.Raw/train/train_splits'
LABEL_CSV = '/kaggle/input/meld-dataset/MELD-RAW/MELD.Raw/train/train_sent_emo.csv'
KNOWN_FACE_DIR = '/kaggle/input/friends-known-faces/known-faces'
OUTPUT_DIR = '/kaggle/working/output_faces'
os.makedirs(OUTPUT_DIR, exist_ok=True)
MATCH_THRESHOLD = 0.7  # cosine similarity threshold

In [21]:
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Initialize models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=True, device=device)
vggface_model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

/usr/local/lib/python3.11/dist-packages/facenet_pytorch/models/mtcnn.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_dict_path)
/usr/local

  0%|          | 0.00/107M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/facenet_pytorch/models/inception_resnet_v1.py:329: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(cached_file)

In [22]:
from PIL import UnidentifiedImageError

known_embeddings = {}

for character in os.listdir(KNOWN_FACE_DIR):
    char_path = os.path.join(KNOWN_FACE_DIR, character)
    if not os.path.isdir(char_path):
        continue

    embeddings = []
    for file in os.listdir(char_path):
        if not file.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
        img_path = os.path.join(char_path, file)
        try:
            img = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error opening {img_path}: {e}")
            continue
        face = transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            embedding = vggface_model(face)
            embeddings.append(embedding)

    if embeddings:
        # Average embeddings to get a robust representation
        embeddings_stack = torch.cat(embeddings, dim=0)
        avg_embedding = torch.mean(embeddings_stack, dim=0)
        known_embeddings[character] = avg_embedding
    else:
        print(f"No embeddings found for {character}")

print(f"Saved {len(known_embeddings)} known embeddings.")

Saved 6 known embeddings.


In [23]:
import torch.nn.functional as F

for name, emb in known_embeddings.items():
    print(f"{name}: {emb.shape}")

F.cosine_similarity(known_embeddings['joey'].unsqueeze(0), known_embeddings['ross'].unsqueeze(0)).item()

phoebe: torch.Size([512])
monica: torch.Size([512])
joey: torch.Size([512])
ross: torch.Size([512])
rachel: torch.Size([512])
chandler: torch.Size([512])


0.4271075427532196

In [24]:
# Predict identity
def identify_face(embedding, threshold=0.7):
    best_match = None
    best_score = -1
    for name, known_embed in known_embeddings.items():
        score = F.cosine_similarity(embedding, known_embed.unsqueeze(0)).item()
        if score > best_score:
            best_score = score
            best_match = name
    if best_score > threshold:
        return best_match, best_score
    else:
        return "unknown", best_score

# Load MELD labels
df = pd.read_csv(LABEL_CSV)
grouped = df.groupby(['Dialogue_ID', 'Utterance_ID'])

In [ ]:
from tqdm import tqdm
import torch.nn.functional as F

CONF_THRESHOLD = 0.7
log_file = open("face_extraction_log.txt", "w")

for (dia_id, utt_id), group in tqdm(grouped, desc='Processing utterances'):
    video_name = f'dia{dia_id}_utt{utt_id}.mp4'
    video_path = os.path.join(VIDEO_DIR, video_name)

    if not os.path.exists(video_path):
        print(f"Video not found: {video_path}")
        continue

    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if frame_count == 0:
        print(f"Skipping video (no frames): {video_path}")
        cap.release()
        continue

    sample_frames = min(5, frame_count)
    step = max(1, frame_count // sample_frames)

    best_score = -1
    best_face_img = None
    best_name = None

    for i in range(0, frame_count, step):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        success, frame = cap.read()
        if not success:
            break

        # Detect faces
        boxes, _ = mtcnn.detect(frame)
        if boxes is None:
            continue

        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            face_img = frame[y1:y2, x1:x2]
            if face_img.size == 0:
                continue

            pil_face = Image.fromarray(cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)).convert('RGB')
            face_tensor = transform(pil_face).unsqueeze(0).to(device)

            with torch.no_grad():
                emb = vggface_model(face_tensor)

            # Identify character (assuming `identify_character` returns name and confidence)
            name, score = identify_face(emb)

            if name != "Unknown" and score > best_score:
                best_score = score
                best_face_img = pil_face
                best_name = name

    cap.release()

    # Save the best face for this utterance
    output_filename = f"result_dia{dia_id}_utt{utt_id}.png"
    output_path = os.path.join(OUTPUT_DIR, output_filename)

    if best_face_img is not None:
        best_face_img.save(output_path)
        if best_score < CONF_THRESHOLD:
            log_file.write(f"{video_name}: saved low-confidence match ({best_name}, {best_score:.2f})\n")
    else:
        log_file.write(f"{video_name}: no face detected\n")

log_file.close()


Processing utterances:   6%|▌         | 593/9989 [25:38<7:57:39,  3.05s/it] 